In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!unzip /content/gdrive/MyDrive/drive-download-20210315T123749Z-001.zip -d /content/gdrive/MyDrive/embedded_data

In [4]:
import json
import numpy as np

In [16]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [5]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Concatenate,LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
filetry = '/content/gdrive/MyDrive/embedded_data/emd_im_name_feature.json'
with open(filetry) as train_file1:
    emd_im_name_feature = json.load(train_file1)
for k,v in emd_im_name_feature.items():
  print(k)
  print((np.array(v)).shape)

In [ ]:
filetry = '/content/gdrive/MyDrive/embedded_data/emd_im_name_article_list_tokenized.json'
with open(filetry) as train_file1:
    emd_im_name_article_list_tokenized = json.load(train_file1)
for k,v in emd_im_name_article_list_tokenized.items():
  print(k)
  print((np.array(v)).shape)

In [ ]:
filetry = '/content/gdrive/MyDrive/embedded_data/emd_im_name_caption_tokenized.json'
with open(filetry) as train_file1:
    emd_im_name_caption_tokenized = json.load(train_file1)
for k,v in emd_im_name_caption_tokenized.items():
  print(k)
  print((np.array(v)).shape)

In [9]:
all_data=[]
for k,v in emd_im_name_feature.items():
  all_data.append(k)

In [10]:
array=[]
for i in range(1500):
  array.append(np.array(emd_im_name_article_list_tokenized[all_data[i]]))
X_article_train=np.stack(array, axis=0)
print(X_article_train.shape)

(1500, 40, 1024)


In [11]:
array=[]
for i in range(1500):
  npp=np.array(emd_im_name_caption_tokenized[all_data[i]])
  npp=npp.flatten()
  array.append(npp)
Y_caption_train=np.stack(array, axis=0)
print(Y_caption_train.shape)

(1500, 1024)


In [12]:
array=[]
for i in range(1500):
  npp=np.array(emd_im_name_feature[all_data[i]])
  npp=npp.flatten()
  array.append(npp)

X_image_train=np.stack(array, axis=0)
print(X_image_train.shape)

(1500, 1024)


In [13]:
input1 = Input(shape=(40,1024))
inp= Dropout(0.2)(input1)
ls = LSTM(1024)(inp)
input2 = Input(shape=(1024,))
con_cat = Concatenate()([ls, input2])
re= Reshape((1, 2048), input_shape=(2048,))(con_cat)
re1= Dropout(0.3)(re)
ls1 = LSTM(1024)(re1)
model = Model(inputs=[input1, input2], outputs=ls1)

In [ ]:
model.summary()

In [17]:
opt = tf.keras.optimizers.SGD(learning_rate=0.6,momentum=0.9)

In [18]:
model.compile(loss='categorical_crossentropy', optimizer=opt)

In [ ]:
model.fit(x=[X_article_train,X_image_train],y=Y_caption_train,epochs=30)

Testing

In [22]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from google.colab.patches import cv2_imshow

In [64]:
filetry = '/content/gdrive/MyDrive/btp_data/im_name_article_list_1.json'
with open(filetry) as train_file1:
    article_list_real = json.load(train_file1)
filetry = '/content/gdrive/MyDrive/btp_data/im_name_caption.json'
with open(filetry) as train_file1:
    caption_real = json.load(train_file1)

In [72]:
filetry = '/content/gdrive/MyDrive/btp_data/im_name_capno_50_final.json'
with open(filetry) as train_file1:
    img_cap_real = json.load(train_file1)

In [38]:
array=[]
for i in range(1505,1600):
  array.append(np.array(emd_im_name_article_list_tokenized[all_data[i]]))
X_article_test=np.stack(array, axis=0)
print(X_article_test.shape)

(95, 40, 1024)


In [39]:
array=[]
for i in range(1505,1600):
  npp=np.array(emd_im_name_feature[all_data[i]])
  npp=npp.flatten()
  array.append(npp)

X_image_test=np.stack(array, axis=0)
print(X_image_test.shape)

(95, 1024)


In [43]:
output=model.predict([X_article_test,X_image_test])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


In [96]:
import cv2

In [98]:
for i in range(1505,1600):
  key=all_data[i]
  context_vec = output[i-1505]
  #print(context_vec.shape)
  m=-2
  cap_final=""
  for j in range(0,len(article_list_real[key])):
    npo = np.array(emd_im_name_article_list_tokenized[key]) 
    B = npo[j]
    cos_sim=cosine_similarity(context_vec.reshape(1,-1),B.reshape(1,-1))
    if cos_sim > m:
      m=cos_sim
      cap_final = article_list_real[key][j]
  img_path='/content/gdrive/MyDrive/Use Image/'+str(key)+'_'+str(img_cap_real[key]) +'.jpg'
  image = cv2.imread(img_path)
  print(cv2_imshow(image))
  print("Caption:")
  print(cap_final)
  print("True val")
  print(caption_real[key])
  print("\n\n")

Output hidden; open in https://colab.research.google.com to view.